# Sahana Thoravalli Prabhuswamy
# Assignment 07 - NLP

Github link for Deep learning assignments.

https://github.com/SahanaTP/DeepLearningAssignments

# Loading required libraries

In [36]:
import gensim.downloader
from tqdm import tqdm
import string

In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from google.colab import drive
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# loading Wikipedia GLoVE Word2Vec

In [ ]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


# Similarity between man and women

In [ ]:
cosine_distance = glove_vectors.similarity('man', 'women')
cosine_distance

0.5303662

# Similarity between chair and throne

In [ ]:
cosine_distance = glove_vectors.similarity('chair', 'throne')
cosine_distance

0.30013406

# Similarity between water and baby

In [ ]:
cosine_distance = glove_vectors.similarity('water', 'baby')
cosine_distance

0.4031727

In [ ]:
# logic behind analogies
v = glove_vectors['king']-glove_vectors['man']+glove_vectors['women']

array([ 0.13780999, -0.31693006, -0.502452  ,  1.0154101 , -0.14744718,
        0.79945   , -0.99088997,  0.27473304, -1.17468   , -1.263679  ,
        0.05625001, -0.30519003, -0.26187   ,  0.56635   , -0.617401  ,
       -1.139032  , -0.494089  , -0.995551  , -1.14527   , -0.00825998,
       -0.71497   , -1.03702   ,  1.1123    ,  1.2681019 , -0.17834002,
       -0.25116703,  1.41917   , -1.48201   ,  1.17335   , -0.397709  ,
        0.728463  ,  0.38424   ,  0.760954  , -0.05114001,  0.30534002,
        0.32726002, -0.10284996,  1.70397   , -1.601243  ,  0.07513499,
       -0.97814   ,  0.334166  , -0.84703   , -1.05203   , -0.306062  ,
       -0.91162   , -0.09182498,  0.510078  ,  0.23192002, -0.82496   ,
       -0.42551002,  0.19180298,  0.34305006,  0.40857995, -0.02853002,
       -1.8389    ,  0.31365   , -0.07905903,  0.76307005, -0.16797   ,
       -0.56675   , -0.16105002,  1.21367   ,  0.089894  ,  0.62215006,
       -0.11048299, -0.79039   ,  0.44873005,  0.30726   , -0.60

# Finding analogies

In [ ]:
result = glove_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
most_similar_key, similarity = result[0]
print(f'{most_similar_key} is to king as Woman is to Man.')

queen is to king as Woman is to Man.


In [ ]:
result = glove_vectors.most_similar(positive=['man', 'princess'], negative=['woman'])
most_similar_key, similarity = result[0]
print(f'{most_similar_key} is to princess as man is to woman.')

prince is to princess as man is to woman.


In [ ]:
result = glove_vectors.most_similar(positive=['child', 'woman'], negative=['adult'])
most_similar_key, similarity = result[0]
print(f'{most_similar_key} is to women as child is to adult.')

mother is to women as child is to adult.


# Spam and Ham classification using Naive bayes and word2vec

# Reading dataset

In [4]:
import pandas as pd

data = pd.read_csv('spam.csv', encoding='latin-1')
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


# checking for class distribution

In [5]:
# checking the distribution of the target variable
data["v1"].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

# Text pre-processing like, punctuation removal, stop word removal and other than english language words removal

In [6]:
# pre processing text
def text_pre_processing(sms):

    # removing punctuations
    sms_wo_punct = [x for x in sms if x not in string.punctuation]
    sms_wo_punct = ''.join(sms_wo_punct)

    # keeping only english letters
    sms_wo_punct_only_words = [x for x in sms_wo_punct if x.isalpha() or x == " "]
    sms_wo_punct_only_words = ''.join(sms_wo_punct_only_words)

    # converting the data to lower case
    sms_wo_punct_only_words_lc = sms_wo_punct_only_words.lower()

    # removing stopwords
    sms_wo_punct_only_words_lc = sms_wo_punct_only_words_lc.split(" ")
    sms_wo_punct_only_words_lc_wo_sw = [x for x in sms_wo_punct_only_words_lc if x not in stopwords.words('english')]
    sms_wo_punct_only_words_lc_wo_sw = ' '.join(sms_wo_punct_only_words_lc_wo_sw)

    return sms_wo_punct_only_words_lc_wo_sw

In [7]:
# pre processed sms
data_processed = pd.DataFrame(columns = ["pre_processed_sms"])
for i in tqdm(data["v2"]):
    data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
data_processed

Streaming output truncated to the last 5000 lines.
<ipython-input-7-cedb96c765ef>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
<ipython-input-7-cedb96c765ef>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
<ipython-input-7-cedb96c765ef>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
<ipython-input-7-cedb96c765ef>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

,pre_processed_sms
0,go jurong point crazy available bugis n great ...
1,ok lar joking wif u oni
2,free entry wkly comp win fa cup final tkts st...
3,u dun say early hor u c already say
4,nah dont think goes usf lives around though
...,...
5567,nd time tried contact u u å pound prize clai...
5568,ì b going esplanade fr home
5569,pity mood soany suggestions
5570,guy bitching acted like id interested buying s...


# Splitting data into train and test

In [8]:
# Splitting into train and test
x_train, x_test, y_train, y_test = train_test_split(data_processed, data["v1"], test_size = 0.20, random_state = 100)
x_train = x_train.reset_index(drop = True)
x_test = x_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [9]:
x_train

,pre_processed_sms
0,ya nowonion roast
1,sen told going join uncle finance cbe
2,sorry ill call later
3,know hows anthony bringing money ive school fe...
4,huh hyde park mel ah opps got confused anyway ...
...,...
4452,work mon thurs sat cant leh booked liao day u ...
4453,ok knackered came home went sleep good full ti...
4454,nah cant help ive never iphone
4455,still looking job much tas earn


In [31]:
words_in_sentences=[]
for i in range(len(x_train)):
    words_in_sentences.append(x_train['pre_processed_sms'].iloc[i])

# Training and building vocabulary for word2vec model

In [32]:
# Using word to vec
alg = Word2Vec( vector_size = 500, min_count = 1)
alg.build_vocab(words_in_sentences)
alg.train(words_in_sentences, total_examples = len(x_train), epochs = alg.epochs)

(204359, 1163415)

In [42]:
# converting words to vector
def convert_word_to_vector(sms, size):
    vec = np.zeros(500).reshape((1, 500))
    count = 0
    for word in sms:
        try:
            vec += alg.wv.get_vector(word).reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [45]:
# converting training and testing to vectors
x_train_vecs = scale(np.concatenate([convert_word_to_vector(z, 500) for z in x_train["pre_processed_sms"]]))
x_test_vecs = scale(np.concatenate([convert_word_to_vector(z, 500) for z in x_test["pre_processed_sms"]]))

In [58]:
from sklearn.preprocessing import MinMaxScaler

# Assuming X is your feature matrix
scaler = MinMaxScaler(feature_range=(0, 1))
x_train_vecs = np.concatenate([convert_word_to_vector(z, 500) for z in x_train["pre_processed_sms"]])
X_train_scaled = scaler.fit_transform(x_train_vecs)

In [59]:
# Train a Multinomial Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_scaled, y_train)

MultinomialNB()

In [60]:
from sklearn.metrics import accuracy_score

In [61]:
x_test_vecs = np.concatenate([convert_word_to_vector(z, 500) for z in x_test["pre_processed_sms"]])
X_test_scaled = scaler.fit_transform(x_test_vecs)

# Evaluating train accuracy and confusion matrix

In [62]:
# training confusion matrix
y_train_pred = clf.predict(X_train_scaled)
accuracy_train = accuracy_score(y_train,y_train_pred)

In [63]:
accuracy_train

0.8456360780794256

In [64]:
confusion_matrix(y_train,y_train_pred)

array([[3581,  274],
       [ 414,  188]])

# Evaluating test accuracy and confusion matrix

In [65]:
# testing confusion matrix
y_test_pred = clf.predict(X_test_scaled)
accuracy_test = accuracy_score(y_test,y_test_pred)
accuracy_test

0.7183856502242153

In [66]:
confusion_matrix(y_test,y_test_pred)

array([[688, 282],
       [ 32, 113]])